In [1]:
# pip install -U langchain-cli

In [2]:
# # adding packages from 
# # https://github.com/langchain-ai/langchain/tree/master/templates
# langchain app add $PROJECT_NAME

# # adding custom GitHub repo packages
# langchain app add --repo $OWNER/$REPO
# # or with whole git string (supports other git providers):
# # langchain app add git+https://github.com/hwchase17/chain-of-verification

# # with a custom api mount point (defaults to `/{package_name}`)
# langchain app add $PROJECT_NAME --api_path=/my/custom/path/rag

In [3]:
pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [4]:
from pinecone import ServerlessSpec
import pinecone as pc


c:\Users\mckal\anaconda3\envs\AI\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [5]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="PINECONE_API_KEY")
index = pc.Index("diamonds")

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'X-Cloud-Trace-Context': '2a24c2c747589562e6132c2ae2e21b80', 'Date': 'Wed, 17 Apr 2024 23:08:02 GMT', 'Content-Type': 'text/html', 'Server': 'Google Frontend', 'Content-Length': '15', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Invalid API Key


In [ ]:
existing_indexes = pc.list_indexes()
print(existing_indexes)


{'indexes': [{'dimension': 1536,
              'host': 'diamonds-c0p28jm.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'diamonds',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}


once it runs comment out 

In [ ]:
# pc.delete_index("diamonds")
# pc.create_index(
#     name="diamonds",
#     dimension=1536,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )


In [ ]:
# to load the environment variables
from dotenv import load_dotenv
import os

# to tokenize, chunk, and embed the text
from openai import OpenAI
import nltk
import tiktoken
from typing import List

# to distill the results
from langchain_openai import ChatOpenAI
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mckal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Load environment variables.
variables_to_clear = ['OPENAI_API_KEY',
                      'LANGCHAIN_TRACING_V2',
                        'LANGCHAIN_ENDPOINT',
                        'LANGCHAIN_API_KEY',
                        'LANGCHAIN_PROJECT']
for var in variables_to_clear:
    if var in os.environ:
        del os.environ[var]

load_dotenv()

True

# Question Answers 
Here are the answers to your diamond-related questions:

## What is the most famous type of diamond cut?

The most famous type of diamond cut is the round brilliant cut. It is known for its ability to maximize a diamond's sparkle and brilliance due to its precise faceting.
## What is the process of diamond certification?

Diamond certification is a process in which a diamond is evaluated and graded based on its characteristics such as carat weight, cut, color, and clarity (known as the 4Cs). This evaluation is conducted by a reputable gemological laboratory, such as the Gemological Institute of America (GIA) or the International Gemological Institute (IGI). A certificate detailing these characteristics is provided to verify the diamond's quality and authenticity.
## Who is the top diamond dealer in the world?

The diamond industry is vast, and there isn't one definitive "top diamond dealer" globally. However, some of the largest and most well-known diamond companies include De Beers, Alrosa, and Rio Tinto. These companies are known for their extensive involvement in diamond mining and trade.
## What are the signs of diamond impurities?

Diamond impurities or inclusions are natural features within a diamond that can affect its clarity. Signs of diamond impurities include internal flaws such as feathers, crystals, or needles. Surface blemishes may include pits or scratches. These imperfections are often evaluated and rated on a clarity scale, ranging from "Flawless" to "Included."
## What are blood diamonds?

Blood diamonds, also known as conflict diamonds, are diamonds mined in war zones and sold to finance armed conflict against governments. The term gained widespread awareness due to humanitarian concerns related to unethical mining practices and exploitation. Efforts such as the Kimberley Process have been established to prevent the trade of conflict diamonds.
## What is the history of the diamond industry?

The diamond industry has a long and storied history. Diamonds were first discovered and mined in India around the 4th century BCE. The industry expanded with the discovery of diamond deposits in South Africa in the late 1800s, which led to the establishment of large diamond mining companies like De Beers. Today, diamonds are sourced from various regions worldwide, including Russia, Australia, and Canada.
## Who are the major players in the diamond market?

Major players in the diamond market include companies involved in diamond mining, trading, and retailing. De Beers, Alrosa, and Rio Tinto are some of the leading diamond mining companies. In terms of retail, companies like Tiffany & Co., Cartier, and Graff Diamonds are well-known luxury brands in the diamond jewelry market.
## How many diamonds are typically used in high-end jewelry?

The number of diamonds used in high-end jewelry can vary depending on the design and size of the piece. For example, a solitaire engagement ring may feature a single prominent diamond, while a necklace or bracelet might include multiple smaller diamonds set in intricate patterns. High-end jewelry pieces often focus on quality and artistry, with diamonds chosen for their exceptional cut, clarity, color, and carat weight.
## What will Langsmith help us learn about diamond appraisals?

Langsmith is likely a reference to an expert or resource specializing in diamond appraisals. An expert like Langsmith can teach us how to evaluate diamonds based on the 4Cs (carat, cut, color, and clarity), how to identify diamond treatments or enhancements, and how to properly assess the value of a diamond. They may also provide guidance on recognizing reputable grading certificates and authenticating diamonds.

In [ ]:
# A large block of text to vectorize as embeddings
text = """The most famous type of diamond cut is the round brilliant cut.
 It is known for its ability to maximize a diamond's sparkle and brilliance due to its precise faceting. 
Diamond certification is a process in which a diamond is evaluated and graded based on its characteristics such as carat weight, cut, color, and clarity (known as the 4Cs). 
This evaluation is conducted by a reputable gemological laboratory, such as the Gemological Institute of America (GIA) or the International Gemological Institute (IGI).
 A certificate detailing these characteristics is provided to verify the diamond's quality and authenticity. The diamond industry is vast, and there isn't one definitive "top diamond dealer" globally. 
 However, some of the largest and most well-known diamond companies include De Beers, Alrosa, and Rio Tinto. These companies are known for their extensive involvement in diamond mining and trade.
Diamond impurities or inclusions are natural features within a diamond that can affect its clarity. Signs of diamond impurities include internal flaws such as feathers, crystals, or needles.
 Surface blemishes may include pits or scratches. These imperfections are often evaluated and rated on a clarity scale, ranging from "Flawless" to "Included."
Blood diamonds, also known as conflict diamonds, are diamonds mined in war zones and sold to finance armed conflict against governments. 
The term gained widespread awareness due to humanitarian concerns related to unethical mining practices and exploitation.
 Efforts such as the Kimberley Process have been established to prevent the trade of conflict diamonds.
The diamond industry has a long and storied history. Diamonds were first discovered and mined in India around the 4th century BCE. 
The industry expanded with the discovery of diamond deposits in South Africa in the late 1800s, which led to the establishment of large diamond mining companies like De Beers. 
Today, diamonds are sourced from various regions worldwide, including Russia, Australia, and Canada.
Major players in the diamond market include companies involved in diamond mining, trading, and retailing. De Beers, Alrosa, and Rio Tinto are some of the leading diamond mining companies. 
In terms of retail, companies like Tiffany & Co., Cartier, and Graff Diamonds are well-known luxury brands in the diamond jewelry market.
The number of diamonds used in high-end jewelry can vary depending on the design and size of the piece. 
For example, a solitaire engagement ring may feature a single prominent diamond, while a necklace or bracelet might include multiple smaller diamonds set in intricate patterns. 
High-end jewelry pieces often focus on quality and artistry, with diamonds chosen for their exceptional cut, clarity, color, and carat weight.
Langsmith is likely a reference to an expert or resource specializing in diamond appraisals. 
An expert like Langsmith can teach us how to evaluate diamonds based on the 4Cs (carat, cut, color, and clarity), how to identify diamond treatments or enhancements, and how to properly assess the value of a diamond. 
They may also provide guidance on recognizing reputable grading certificates and authenticating diamonds.
   """

## chat-gpt on embedding: 
The model identifier "text-embedding-3-small" is not a recognized model according to my last knowledge update in January 2022, and it does not correspond to an embedding model offered by OpenAI at the time. OpenAI's notable embedding models include "text-embedding-ada-002" and "text-embedding-ada-001." These are used for generating text embeddings, which are vector representations of text that capture semantic meaning.

You might want to consider using "text-embedding-ada-002" instead of "text-embedding-3-small" if you're working with OpenAI embeddings. This model is known for its efficient performance and accuracy in generating text embeddings:

In [ ]:
# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
EMBED_MODEL = "text-embedding-ada-002"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)
MAX_TOKENS = 1536

def prep(text: str):
    return text.replace("\n", " ").replace("\r", " ").replace("\t", " ")

def tokenize(text: List[str]):
    encoding = tiktoken.encoding_for_model(EMBED_MODEL)
    return encoding.encode(text)

def embed(tokens: List[int]):
    response = client.embeddings.create(input=tokens,model=EMBED_MODEL)
    return response.data[0].embedding

def chunk_text(text:str):
    current_chunk = []
    current_para = ""
    chunks = []
    paras = []
    current_len = 0
    sentences = nltk.sent_tokenize(text)
    chunks_of_tokens = []
    
    for sentence in sentences:
        # Tokenize the sentence
        sentence_tokens = tokenize(sentence)
        sentence_token_len = len(sentence_tokens)
        
        # Check if adding the next sentence exceeds the max token limit
        if current_len + sentence_token_len > MAX_TOKENS:
            # Add the current chunk to the list and start a new one
            paras.append(current_para)
            current_para = ""
            chunks_of_tokens.append(current_chunk)
            embeddings = embed(current_chunk)
            chunks.append(embeddings)
            current_chunk = []
            current_len = 0
        
        # Add the sentence to the current chunk
        current_para += " " + sentence
        current_chunk.extend(sentence_tokens)
        current_len += sentence_token_len
    
    # Add the last chunk if it's not empty
    if current_chunk:
        paras.append(current_para)
        chunks_of_tokens.append(current_chunk)
        embeddings = embed(current_chunk)
        chunks.append(embeddings)

    return paras, chunks, chunks_of_tokens

def create_embeddings(filename: str):
    with open(filename, "r") as file:
        text = file.read()
    text = prep(text)
    return chunk_text(text)
    
def create_embeddings_prompt(prompt:str):
    prompt = prep(prompt)
    return chunk_text(prompt)

def vectorize_chunks(paras: List, chunks: List, **kwargs):
    vectors = []
    for i in range(len(chunks)):
        if "filename" in kwargs:
            vectors.append({"id": f"{i}", "values": chunks[i], "metadata": {"file": filename, "para": f"{paras[i]}"}})
        else:
            vectors.append({"id": f"{i}", "values": chunks[i], "metadata": {"para": f"{paras[i]}"}})
        
    return vectors

In [ ]:
paras, chunks, chunks_of_tokens  = create_embeddings_prompt(text)
vectors = vectorize_chunks(paras, chunks)

In [ ]:
print(len(paras), len(paras[0]), len(chunks), len(chunks[0]), len(chunks_of_tokens), len(chunks_of_tokens[0]), len(vectors))
print(paras[0])
print(chunks[0])
print(chunks_of_tokens[0])
print(vectors[0])

1 3190 1 1536 1 620 1
 The most famous type of diamond cut is the round brilliant cut. It is known for its ability to maximize a diamond's sparkle and brilliance due to its precise faceting. Diamond certification is a process in which a diamond is evaluated and graded based on its characteristics such as carat weight, cut, color, and clarity (known as the 4Cs). This evaluation is conducted by a reputable gemological laboratory, such as the Gemological Institute of America (GIA) or the International Gemological Institute (IGI). A certificate detailing these characteristics is provided to verify the diamond's quality and authenticity. The diamond industry is vast, and there isn't one definitive "top diamond dealer" globally. However, some of the largest and most well-known diamond companies include De Beers, Alrosa, and Rio Tinto. These companies are known for their extensive involvement in diamond mining and trade. Diamond impurities or inclusions are natural features within a diamond t

In [ ]:
import os
from pinecone import Pinecone

# Retrieve the API key from environment variables
api_key = os.getenv("PINECONE_API_KEY")
if not api_key:
    raise ValueError("PINECONE_API_KEY environment variable is not set.")

# Create Pinecone client
pc = Pinecone(api_key=api_key)

# Attempt to access the index
try:
    index = pc.Index("diamonds")
    print("Successfully accessed the index 'diamonds'.")
except Exception as e:
    print(f"Error accessing the index 'diamonds': {e}")


Successfully accessed the index 'diamonds'.


In [ ]:
# from pinecone import Pinecone DIDNT WORK

# pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# index = pc.Index("ai-class1")

In [ ]:
filename = "README.md"
paras, chunks, chunks_of_tokens = create_embeddings(filename)
vectors_from_file = vectorize_chunks(paras, chunks, filename=filename)
vectors.extend(vectors_from_file)
# filename = "resources/README copy.md"
# paras, chunks, chunks_of_tokens = create_embeddings(filename)
# vectors_from_file = vectorize_chunks(paras, chunks, filename=filename)
# vectors.extend(vectors_from_file)

index.upsert(
    vectors=vectors     
)1

{'upserted_count': 1}

In [ ]:
print(len(vectors[0]["values"]))


1536


In [ ]:
query_responses=[]

def ask_a_question(prompt):
    # convert the prompt to chunks of  embeddings
    paras, chunks, chunks_of_tokens  = create_embeddings_prompt(prompt)
    print(f"Embeddings: {chunks[0]}")
    # vectorize the embeddings
    prompt_vectors = vectorize_chunks(paras, chunks)
    print(f"Vectorized: {prompt_vectors[0]}")
    # search the index for the best match using semantic search
    query_response = index.query(
        top_k=2,
        vector=prompt_vectors[0]["values"]
    )
    query_responses.append(query_response)
    print(f"Query response: {query_response}")
    # get the id of the best match
    best_id = query_response["matches"][0]["id"]
    print(f"Best ID: {best_id}")
    # fetch the best match from the index
    result = index.fetch(ids=[best_id])
    # get the paragraph of interest from the result metadata
    para_of_interest = result["vectors"][best_id]["metadata"]["para"]
    print(f"Para of interest: {para_of_interest}")
    # Initialize the langchain chat model.
    llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.0)
    # turn the para_of_interest into a Document
    document = Document(page_content=para_of_interest)
    # Create the QA chain using the LLM.
    chain = load_qa_chain(llm)
    # Pass the para_of_interest and the prompt to the chain, and print the result.
    question = "If you can't find the answer in the provided document, say, I just don't know the answer to that, otherwise, answer the question. " + prompt
    result = chain.invoke({"input_documents": [document], "question": question})
    return result["output_text"]

## Questions 
What is the most famous type of diamond cut?
What is the process of diamond certification?
Who is the top diamond dealer in the world?
What are the signs of diamond impurities?
What are blood diamonds?
What is the history of the diamond industry?
Who are the major players in the diamond market?
How many diamonds are typically used in high-end jewelry?
What will Langsmith help us learn about diamond appraisals?

In [ ]:
query_responses=[]

questions = ["What is the most famous type of diamond cut?",
            "What is the process of diamond certification?",
            "Who is the top diamond dealer in the world?",
            "What are the signs of diamond impurities?",
            "What are blood diamonds?",
            "What is the history of the diamond industry?",
            "Who are the major players in the diamond market?",
            "How many diamonds are typically used in high-end jewelry?",
            "What will Langsmith help us learn about diamond appraisals?"]

answers = []
for question in questions:
    answers.append(ask_a_question(question))

Embeddings: [-0.015503785572946072, 0.007622694596648216, 0.005119479261338711, -0.01576218195259571, -0.010549033991992474, 0.018940458074212074, -0.029767267405986786, 0.00627903314307332, -0.009108473546802998, -0.008656280115246773, 0.01166013814508915, 0.012176930904388428, -0.00415049260482192, -0.00819762609899044, -0.011821635998785496, 0.024534739553928375, 0.02689906768500805, 0.011123965494334698, 0.018449503928422928, -0.009147233329713345, -0.03971552848815918, 0.005830069072544575, 0.01290690153837204, -0.021317705512046814, -0.006585878785699606, -0.004819093272089958, 0.029198795557022095, -0.012642044574022293, -0.005830069072544575, -0.022583847865462303, 0.018307385966181755, 0.0030490776989609003, -0.03325561806559563, -0.001703801448456943, -0.01822986826300621, 0.01832030527293682, 7.797314174240455e-05, 0.002420851495116949, 0.01162783894687891, 0.005035500042140484, 0.006976703181862831, -0.008701499551534653, 5.788685302832164e-05, -0.0032234953250736, -0.00841

IndexError: list index out of range

In [ ]:
ix = 0
for query_response in query_responses:
    print(f"Match Score: {query_response['matches'][0]['score']}")
    print(f"Question: {questions[ix]}")
    print(f"Answer:   {answers[ix]}\n\n")
    ix += 1

In [ ]:
#  Uncomment the line below if you are using Google Colab.
# !pip install transformers

In [ ]:
# Import the pipeline class from the transformers module. 
from transformers import pipeline

In [ ]:
# Use the text-generation parameter for the pipeline and EleutherAI/gpt-neo-1.3B model. 
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')

: 

pad token id is a hyper parameter 

In [ ]:
# Give the model a prompt. 
prompt = "I like gardening because"
# Pass the prompt to the generator
results = generator(prompt, max_length=125, pad_token_id=50256)
# Get the text based on the prompt. 
generated_text = results[0]['generated_text']
# Print the generated text.
print(generated_text)

In [ ]:
# Use the text-generation parameter for the pipeline and EleutherAI/gpt-neo-125m model. 
small_generator = pipeline('text-generation', model='EleutherAI/gpt-neo-125m')

In [ ]:
# Give the model a prompt. 
prompt = "My favorite animal is the cat because "
# Pass the prompt to the generator. Use `max_length=25`.
new_results = small_generator(prompt, max_length=25, pad_token_id=50256)
# Get the text based on the prompt. 
generated_text = new_results[0]['generated_text']
# Print the generated text.
print(generated_text)

My favorite animal is the cat because ive never seen a cat before. ive never seen a cat before. ive
